In [1]:
import numpy as np
import random
import math
from tqdm import tqdm
import cmath

In [2]:
def TS_cha_gen(iteration, Es_dBm, rho_B, rho_E, Noise_Var_B, Noise_Var_E, R_T, R_E):
    R_S = R_T - R_E
    Es = 10**(Es_dBm/10)/1000
    sigma_E = math.sqrt(Noise_Var_E)
    sigma_B = math.sqrt(Noise_Var_B)
    avg_SNR_B = (2*Es) / (sigma_B)**2
    avg_SNR_E = (2*Es) / (sigma_E)**2
    Noise_Var_E_v = 1 - rho_E**2
    Noise_Var_B_v = 1 - rho_B**2    
    
    count_ConOutage_LML = np.zeros(iteration)
    count_ConOutage_ZF  = np.zeros(iteration)
    count_ConOutage_DF  = np.zeros(iteration)

    count_SecOutage_LML = np.zeros(iteration)
    count_SecOutage_ZF  = np.zeros(iteration)
    count_SecOutage_DF  = np.zeros(iteration)

    count_LML_LML = np.zeros(iteration)
    count_LML_DF = np.zeros(iteration)
    count_DF_LML = np.zeros(iteration)
    count_DF_DF = np.zeros(iteration)
    
    for k in tqdm(range(iteration)):
        ####################################
        # Time Selective Channel Generation
        ####################################
        H_B1 = np.divide(np.random.randn(2,1) + 
                np.multiply(1j,np.random.randn(2,1)), math.sqrt(2))
        H_B2 = np.multiply(rho_B, H_B1) + np.divide(np.multiply((np.random.randn(2,1) +
                 1j*np.random.randn(2,1)), cmath.sqrt(Noise_Var_B_v)), math.sqrt(2))
        H_B3 = np.c_[H_B1, H_B2]
        H_B3_JML = np.c_[H_B1, H_B1]
        H_B = ([H_B3[0,0], H_B3[1,0]],
                [np.conjugate(H_B3[1,1]), -np.conjugate(H_B3[0,1])])
        G_B = np.multiply(np.transpose(H_B), H_B)
        H_B_JML = ([H_B3_JML[0,0], H_B3_JML[1,0]],
                [np.conjugate(H_B3_JML[1,1]), -np.conjugate(H_B3_JML[0,1])])
        G_B_JML= np.multiply(np.transpose(H_B_JML), H_B_JML)

        H_E1 = np.divide(np.random.randn(2,1) + 
                np.multiply(1j,np.random.randn(2,1)), math.sqrt(2))
        H_E2 = np.multiply(rho_E, H_E1) + np.divide(np.multiply((np.random.randn(2,1) +
                 1j*np.random.randn(2,1)), cmath.sqrt(Noise_Var_E_v)), math.sqrt(2))
        H_E3 = np.c_[H_E1, H_E2]
        H_E3_JML = np.c_[H_E1, H_E1]
        H_E = ([H_E3[0,0], H_E3[1,0]],
                [np.conjugate(H_E3[1,1]), -np.conjugate(H_E3[0,1])])
        G_E = np.multiply(np.transpose(H_E), H_E)
        H_E_JML = ([H_E3_JML[0,0], H_E3_JML[1,0]],
                [np.conjugate(H_E3_JML[1,1]), -np.conjugate(H_E3_JML[0,1])])
        G_E_JML= np.multiply(np.transpose(H_E_JML), H_E_JML)

        ############################################
        # Effective rerceived SNR for each antenna
        ############################################
        # LML at Bob
        B_Eff_SNR_Ant1_LML = avg_SNR_B / ((1-rho_B**2)*avg_SNR_B + 2) * G_B[0,0]
        B_Eff_SNR_Ant2_LML = avg_SNR_B / ((1-rho_B**2)*avg_SNR_B + 2) * G_B[1,1]

        # LML at Eve
        E_Eff_SNR_Ant1_LML = avg_SNR_E / ((1-rho_E**2)*avg_SNR_E + 2) * G_E[0,0]
        E_Eff_SNR_Ant2_LML = avg_SNR_E / ((1-rho_E**2)*avg_SNR_E + 2) * G_E[1,1]

        # ZF at Bob
        print(G_B[0,0]*G_B[1,1] - abs(G_B[0,1])**2)
        B_zeta = cmath.sqrt(G_B[0,0]*G_B[1,1] - abs(G_B[0,1])**2)   
        B_Eff_SNR_Ant1_ZF = B_zeta**2 / (2*G_B[1,1]) * avg_SNR_B
        B_Eff_SNR_Ant2_ZF = B_zeta**2 / (2*G_B[0,0]) * avg_SNR_B
        
        # ZF at Eve
        E_zeta = cmath.sqrt(G_E[0,0]*G_E[1,1] - abs(G_E[0,1])**2)
        E_Eff_SNR_Ant1_ZF = E_zeta**2 / (2*G_E[1,1]) * avg_SNR_E
        E_Eff_SNR_Ant2_ZF = E_zeta**2 / (2*G_E[0,0]) * avg_SNR_E
        
        # DF at Bob
        B_zeta = cmath.sqrt(G_B[0,0]*G_B[1,1] - abs(G_B[0,1])**2)   
        B_Eff_SNR_Ant1_DF = B_zeta**2 / (2*G_B[1,1]) * avg_SNR_B  # ZF
        B_Eff_SNR_Ant2_DF = avg_SNR_B / 2 * G_B_JML[1,1]          # JML
        
        # DF at Eve
        E_zeta = cmath.sqrt(G_E[0,0]*G_E[1,1] - abs(G_E[0,1])**2)
        E_Eff_SNR_Ant1_DF = E_zeta**2 / (2*G_E[1,1]) * avg_SNR_E   # ZF
        E_Eff_SNR_Ant2_DF = avg_SNR_E / 2 * G_E_JML[1,1]           # JML

        C_B_LML = 1/2*(cmath.log(1+B_Eff_SNR_Ant1_LML, 2) + cmath.log(1+B_Eff_SNR_Ant2_LML, 2))  # Capacity of LML at Bob
        C_B_ZF  = 1/2*(cmath.log(1+B_Eff_SNR_Ant1_ZF, 2) + cmath.log(1+B_Eff_SNR_Ant2_ZF, 2))    # Capacity of ZF at Bob
        C_B_DF  = 1/2*(cmath.log(1+B_Eff_SNR_Ant1_DF, 2) + cmath.log(1+B_Eff_SNR_Ant2_DF, 2))    # Capacity of DF at Bob

        C_E_LML = 1/2*(cmath.log(1+E_Eff_SNR_Ant1_LML, 2) + cmath.log(1+E_Eff_SNR_Ant2_LML, 2))  # Capacity of LML at Eve
        C_E_ZF  = 1/2*(cmath.log(1+E_Eff_SNR_Ant1_ZF, 2) + cmath.log(1+E_Eff_SNR_Ant2_ZF, 2))    # Capacity of ZF at Eve
        C_E_DF  = 1/2*(cmath.log(1+E_Eff_SNR_Ant1_DF, 2) + cmath.log(1+E_Eff_SNR_Ant2_DF, 2))    # Capacity of DF at Eve

        if (C_B_LML <= R_T).all:
                count_ConOutage_LML[k] = 1
        else:
                count_ConOutage_LML[k] = 0

        if (C_B_ZF <= R_T).all:
                count_ConOutage_ZF[k] = 1
        else:
                count_ConOutage_ZF[k] = 0

        if (C_B_DF <= R_T).all:
                count_ConOutage_DF[k]  = 1
        else:
                count_ConOutage_DF[k]  = 0

        if (C_E_LML > R_E).all:
                count_SecOutage_LML[k] = 1
        else:
                count_SecOutage_LML[k] = 0

        if (C_E_ZF > R_E).all:
                count_SecOutage_ZF[k] = 1
        else:
                count_SecOutage_ZF[k] = 0
        
        if (C_E_DF > R_E).all:
                count_SecOutage_DF[k] = 1
        else:
                count_SecOutage_DF[k] = 0         
    
    ConOutage_LML = np.sum(count_ConOutage_LML)/iteration
    ConOutage_ZF  = np.sum(count_ConOutage_ZF) /iteration
    ConOutage_DF  = np.sum(count_ConOutage_DF) /iteration

    SecOutage_LML  = np.sum(count_SecOutage_LML) /iteration
    SecOutage_ZF   = np.sum(count_SecOutage_ZF) /iteration
    SecOutage_DF   = np.sum(count_SecOutage_DF) /iteration

    return ConOutage_LML, ConOutage_ZF, ConOutage_DF, SecOutage_LML, SecOutage_ZF, SecOutage_DF

In [3]:
n = 100
R_T_min = 0
R_T_max = 8
R_T_stepsize = (R_T_max - R_T_min) / n

R_E_min = 0
R_E_max = 8
R_E_stepsize = (R_E_max - R_E_min) / n

iteration = 10^4

rho_B = 0.95 # Time Correlation of Bob's Channel
rho_E = 0.95 # Time Correlation of Eve's Channel

Noise_Var_B = 1  # Bob's Noise Variance
Noise_Var_E = 1  # Eve's Noise Variance

Es_dBm = 40

R_T = np.arange(R_T_min, R_T_max, R_T_stepsize)
R_E = np.arange(R_E_min, R_E_max, R_T_stepsize)

R_T_th = np.power(2, R_T) - 1
R_E_th = np.power(2, R_E) - 1

nData = len(R_T)

Num_ConOutage_LML = np.zeros(nData)
Num_ConOutage_ZF  = np.zeros(nData)
Num_ConOutage_DF  = np.zeros(nData)
Num_SecOutage_LML = np.zeros(nData)
Num_SecOutage_ZF  = np.zeros(nData)
Num_SecOutage_DF  = np.zeros(nData)

for i in tqdm (range (nData)):
    R_T_sample = R_T[i]
    R_E_sample = R_E[i]

    Num_ConOutage_LML[i], Num_ConOutage_ZF[i], Num_ConOutage_DF[i], Num_SecOutage_LML[i], Num_SecOutage_ZF[i], Num_SecOutage_DF[i] = \
        TS_cha_gen(iteration, R_T_sample, R_E_sample, rho_B, rho_E, Es_dBm, R_T, R_E)


# print(R_T)
# print(len(R_T))

100%|██████████| 14/14 [00:00<00:00, 4781.39it/s]


(0.20256733488094825-2.94730455048765j)
(-0.13602014236360005+0.35435001534022675j)
(-0.4528938822176185+0.17325918756095807j)
(0.06151962189174273+0.13311343175795032j)
(0.07855332329534317+0.11757293878098017j)
(-1.892581883321842+1.2624836812338835j)
(0.8054508750132614+1.2839348200054166j)
(-3.496532061139367+0.6709211634432547j)
(-0.31506748585573374+0.05867670945155978j)
(-1.4962767276522828-0.5605126466215513j)
(-0.9643283493640397-0.09717832976923574j)
(-0.5302078853250232-0.28579206780030647j)
(0.21103162967813066+0.023952570785139737j)
(-1.9498340845130224-0.2844851163059356j)


100%|██████████| 14/14 [00:00<00:00, 3585.97it/s]


(-2.2233788988437064-1.6260516493006647j)
(-0.448483993656042+0.05331289782149358j)
(0.7999535552602348-0.49786832706372247j)
(-0.4869929375470091+0.005745426817649814j)
(-0.5835868444967853+0.2609203140928519j)
(-0.7905572119446587-0.7402077824993409j)
(-0.20628215950417023-0.022998430296069006j)
(0.02344310799961731+0.007776968599049538j)
(-0.09339696862127721-0.00023476954762585464j)
(-0.3027392140744105-0.028788509963146976j)
(-16.531978772671646-3.5671265503514897j)
(-1.2384256495244377-0.01809427228655336j)
(-0.04707182543201935-0.09135419458751994j)
(-0.09331646446144864-0.04035867048282778j)


100%|██████████| 14/14 [00:00<00:00, 4821.04it/s]


...

(-163.98639050733345-80.49967239284419j)
(-132.1159429502963+86.30006727902534j)
(3.536965727184522+7.320718102602571j)
(-171.2736322246905-0.06608858535257132j)
(182.8659665521048+201.66402616235786j)
(-187.69647474336213-11.25207884887654j)
(-280.3076551859101+71.80539167870118j)


100%|██████████| 14/14 [00:00<00:00, 1593.41it/s]


(-472.6157950800788+3.4966823594443j)
(-98.11715760784182-38.80615779091433j)
(-413.4386165667274+13.265742224548863j)
(86.9362767601333+57.52209805535559j)
(976.5639163603498-21.139467422951128j)
(-66.86930418270786+8.738163064951205j)
(-11.682475759361292-4.0003377240282605j)


(-2.923505632111591+133.98360656080328j)
(-141.718889937609-9.812576600319147j)
(-680.5593212119622-163.12732901653203j)
(3.128214554438216+46.29175572700363j)
(-390.88960460627675+52.971372039027045j)
(1038.1181430210293-248.74983882917368j)
(-125.11954190246931+0.6917345495375512j)
(-263.83141172642854+48.741022930523286j)
(-0.11938087779047632+0.10454439425888237j)
(-568.3442832466358+10.004250521089787j)
(172.10669627786206+70.78422098545391j)
(-1205.7417588434437+2.412076743439969j)
(-2.183719328927843-16.940515159199883j)
(-106.82196895600768-2.2278132655490497j)


100%|██████████| 100/100 [00:00<00:00, 179.76it/s]


In [4]:
np.shape(R_T)

(100,)

In [11]:
Num_Bob_Pr_LML = np.zeros((100,100))
Num_Bob_Pr_ZF = np.zeros((100,100))
Num_Bob_Pr_DF = np.zeros((100,100))
Num_Eve_Pr_LML = np.zeros((100,100))
Num_Eve_Pr_ZF = np.zeros((100,100))
Num_Eve_Pr_DF = np.zeros((100,100))
R_S = np.zeros((100,100))

for Bob in range(100):
    for Eve in range(100):
        if Bob > Eve:
            R_S[Eve, Bob] = R_T[Bob] - R_E[Eve]
        else:
            R_S[Eve, Bob] = 0
    Num_Bob_Pr_LML = np.c_[Num_ConOutage_LML]
    Num_Bob_Pr_DF = np.c_[Num_ConOutage_DF]

    Num_Eve_Pr_LML = np.c_[np.transpose(Num_SecOutage_LML)]
    Num_Eve_Pr_DF = np.c_[np.transpose(Num_SecOutage_DF)]
    
    Num_Bob_Pr_ZF = np.c_[Num_ConOutage_ZF]
    Num_Eve_Pr_ZF = np.c_[np.transpose(Num_SecOutage_ZF)]

Num_Bob_LML_Eve_LML = np.multiply(R_S, np.multiply((1-Num_Bob_Pr_LML) ,(1-Num_Eve_Pr_LML)))
Num_Bob_LML_Eve_ZF = np.multiply(R_S, np.multiply((1-Num_Bob_Pr_LML), (1-Num_Eve_Pr_ZF)))
Num_Bob_LML_Eve_DF = np.multiply(R_S, np.multiply((1-Num_Bob_Pr_LML), (1-Num_Eve_Pr_DF)))

Num_Bob_ZF_Eve_LML = np.multiply(R_S, np.multiply((1-Num_Bob_Pr_ZF), (1-Num_Eve_Pr_LML)))
Num_Bob_ZF_Eve_ZF = np.multiply(R_S, np.multiply((1-Num_Bob_Pr_ZF), (1-Num_Eve_Pr_ZF)))
Num_Bob_ZF_Eve_DF = np.multiply(R_S, np.multiply((1-Num_Bob_Pr_ZF), (1-Num_Eve_Pr_DF)))

Num_Bob_DF_Eve_LML = np.multiply(R_S, np.multiply((1-Num_Bob_Pr_DF), (1-Num_Eve_Pr_LML)))
Num_Bob_DF_Eve_ZF = np.multiply(R_S, np.multiply((1-Num_Bob_Pr_DF), (1-Num_Eve_Pr_ZF)))
Num_Bob_DF_Eve_DF = np.multiply(R_S, np.multiply((1-Num_Bob_Pr_DF), (1-Num_Eve_Pr_DF)))

Num_Bob_DF_Eve_DF.shape

(100, 100)

In [ ]:
rho_B = 0.95
Noise_Var_B = 1
Noise_Var_B_v = 1 - rho_B**2

H_B1 = np.divide(np.random.randn(2,1) + np.multiply(1j,np.random.randn(2,1)), math.sqrt(2))
H_B2 = np.multiply(rho_B, H_B1) + np.divide(np.multiply((np.random.randn(2,1) + 1j*np.random.randn(2,1)),
        math.sqrt(Noise_Var_B_v)), math.sqrt(2))
H_B3 = np.c_[H_B1, H_B2]
H_B3_JML = np.c_[H_B1, H_B1]
H_B = [[H_B3_JML[0,0], H_B3_JML[1,0]],
         [np.conjugate(H_B3_JML[1,1]), -np.conjugate(H_B3_JML[0,1])]]
print(np.shape(H_B))